In [1]:
import sys
import time
#!{sys.executable} -m pip install inquirer 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from itertools import product 
import pandas as pd
import os
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
import numpy as np
import itertools
import pathlib
import cv
import random
import re


In [2]:
def read_code_from_file():
    global code_file_name
    code_files = [f for f in pathlib.Path(os.path.join(os.getcwd(),"codes")).glob("*")]
    options=[str(s+1) for s in range(len(code_files))]
    options_text='\n'.join(["Enter {} to select {}".format(i+1,f.name)   for i,f in enumerate(code_files)])
    prompt="Please enter index for file in list or full path to file with code: "
    message="{}\n{}\n\n ".format(prompt,options_text)    
    while True:
        response=input(message)
        if response not in options:
            try :
                codefile = response
                f=open(codefile,"r")
                code = f.read()
                f.close()
                break
            except: 
                print("File not found, please try again")
                continue
        else:
            try :
                codefile=code_files[int(response)-1]
                f=open(codefile,"r")
                code = f.read()
                f.close()
                break
            except Exception as e:
                print(e)
                continue
    code_file_name=pathlib.Path(codefile).stem
    code_split=[re.search(r"\[(.*?)]", l).group(1) for l in re.findall(r"(\[[^\[\]]*])", code)]
    param_names = [[l, code_split [i+1]] for i,l in enumerate(code_split) if i in [i-1 for i,l in enumerate(code_split) if '{' in l]]    
    return code,param_names
def get_time_range_params(start,end):
    Months=pd.date_range(start =start, end = end,freq=pd.offsets.MonthBegin(1)).astype(str).tolist()
    params =[(v.split('-')[1],v.split('-')[0]) for v in Months]
    return params 
def all_codes():
    code, param_names = read_code_from_file()
    print('Parameters in code are: {}'.format(param_names))
    v2 = '[NEMSIS Enhanced]' in code
    
    if v2: 
        print('Please enter start month and end month a date between 2014-01-01 and 2016-12-01')
        start=input("Enter start month:(YYYY-MM-01)")
        end=input("Enter end month:(YYYY-MM-01)")
        if not start: start='2014-01-01'
        if not end: end='2016-12-01'
    else:
        print('Please enter start month and end month a date between 2017-01-01 and current month')
        start=input("Enter start month:(YYYY-MM-01)")
        end=input("Enter end month:(YYYY-MM-01)")
        if not start: start = '2017-01-01'
        if not end: end = datetime.today().date()         
    months=get_time_range_params(start,end)
    codes=[]
    for m in months:
        m_code=code
        for i,p in enumerate(m):
            m_code=m_code.replace('{'+str(i)+'}',p)       
        codes.append(m_code)
    return codes, v2

In [3]:
def create_driver():
    driver = webdriver.Chrome()
    driver.get("https://rp.nemsis.org/reportportal/login.aspx?redirect=/rp.nemsis.org/reportportal/olap/MdxDesign.aspx")
    return driver
def login():
    username = driver.find_element_by_name("txtUserName")
    password = driver.find_element_by_name("txtPassword")
    username.clear()
    password.clear()
    username.send_keys("nemsispublic")
    password.send_keys("Nemsis@tac")
    driver.find_element_by_name("btnLogin").click()
def connect_olap():
    driver.find_element_by_id("idDesignMenu").click()
    driver.find_element_by_xpath(".//div[contains(@onclick, 'MdxDesign.aspx')]").click()
def get_iframes():
    return [f.get_attribute('name') for f in driver.find_elements_by_tag_name("iframe")]
def select_table(v2):
    ##select name of element corresponding to tables
    table_id='h3' if v2 else 'h27'
    ShowHideConnect=driver.find_element_by_xpath("//*[contains(@onclick,'ShowHideConnect')]")
    ShowHideConnect.click()
    ifs= get_iframes()
    try: #slow to switch to iframe
        while 'popupFrame1' in ifs: driver.switch_to.frame('popupFrame1') 
    except: pass
    elem = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, table_id)))
    ActionChains(driver).move_to_element(elem).click().perform()
    driver.find_element_by_name("btnOk").click()
    driver.switch_to.parent_frame()
    ##Hide Popup if present
    popup=driver.find_elements_by_id('popupContainer1')
    if popup:driver.execute_script("arguments[0].style.display = 'none'; return arguments[0];", popup[0])        
def activate_mdx_textbox():
    showMdx=driver.find_elements_by_id("idMdx")
    driver.execute_script("arguments[0].style.display = 'block'; return arguments[0];", showMdx[0])
def init_sess(driver,v2=True):
    login()
    connect_olap()
    select_table(v2)
    activate_mdx_textbox()


In [4]:
def repeat(codes):
    started=datetime.today()
    rounds=len(codes)
    print('Start cycle: ',started.strftime("%Y-%m-%d %H:%M:%S"))
    print("new cycle")
    for i,code in enumerate(codes):
        getdata(code)
        print(rounds-i,end=',')
    print('0')
    ended=datetime.today()
    print('End cycle: ' ,ended.strftime("%Y-%m-%d %H:%M:%S"))
    print('cycle duration: ', (ended-started).total_seconds()//60)

def send_code_to_mdx_text_box(text):
    text=text.replace('\n','\\n')
    driver.switch_to.parent_frame()
    textbox=driver.find_element_by_id("txtMDXQuery")    
    textbox.clear()
    arg="arguments[0].value='{}';".format(text)
    driver.execute_script(arg, textbox) 

def excecute_query():
    elem = WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.NAME, "btnExecute")))
    ActionChains(driver).move_to_element(elem).click().perform()
    WebDriverWait(driver, 60).until(EC.invisibility_of_element_located((By.ID, "idAbort")))



def too_many_rows_message():
    ##switch to parent iframe as preventive measure
    driver.switch_to.parent_frame()
    ##if popup container is visible, we got too many rows
    try: 
        WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.ID,'popupFrame1')))
        ifs= get_iframes()
        try: #slow to switch to iframe
            while 'popupFrame1' in ifs: driver.switch_to.frame('popupFrame1') 
        except: pass
        #wait for button OK to be visible then click on it
        elem=WebDriverWait(driver, 50).until(EC.element_to_be_clickable((By.XPATH, ".//button[contains(@onclick,'CloseRet(1);')]")))
        ActionChains(driver).move_to_element(elem).click().perform()
        ##switch back to parent iframe
        driver.switch_to.parent_frame()
        ##Hide Popup if present otherwise the export button will be hidden
        popup=driver.find_elements_by_id('popupContainer1')
        if popup:driver.execute_script("arguments[0].style.display = 'none'; return arguments[0];", popup[0])
    except: pass

def load_more_rows(): 
    elem=driver.find_elements_by_id("idLoadMore")
    while elem: 
        #print(elem)
        ActionChains(driver).move_to_element(elem[0]).click().perform()
        elem=driver.find_elements_by_id("idLoadMore")
        
def export():
    #wait until we see results in table
    #WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.ID, 'htr1')))
    ##If warning messages are covering the button ,
    messages=driver.find_elements_by_class_name("jq-toast-single")
    [driver.execute_script("arguments[0].style.display = 'none'; return arguments[0];", m) for m in messages]
    
    WebDriverWait(driver, 50).until(EC.invisibility_of_element_located((By.CLASS_NAME, "modal-backdrop")))
    WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.ID, "tdShowExport"))).click()
    element = WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.ID, 'idExportDropdown')))
    WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.XPATH, ".//div[contains(@onclick, 'csv')]"))).click()
def close_warning_messages():
    message = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, "jq-toast-single")))
    messages=driver.find_elements_by_class_name("jq-toast-single")
    [driver.execute_script("arguments[0].style.display = 'none'; return arguments[0];", m) for m in messages]
        
        
        
def getdata(code):
    send_code_to_mdx_text_box(code)
    excecute_query()
    too_many_rows_message()
    load_more_rows()
    export()
    close_warning_messages()  
    

In [5]:
def pick_folder_with_files():
    print('pick_folder_with_files():')
    global combined_file_name
    home = pathlib.Path.home()
    downloads_folder=os.path.join(home,"Downloads")
    while True:
        candidate_folders=[f for f in pathlib.Path(downloads_folder).glob("*") if code_file_name in f.name and not f.is_file()]
        options=[str(s+1) for s in range(len(candidate_folders))]
        options_text='\n'.join(["Enter {} to select {}".format(i+1,f.name)   for i,f in enumerate(candidate_folders)])
        prompt="Please enter index for folder in list or full path to folder: "
        message="{}\n{}\n\n ".format(prompt,options_text)
        response=input(message)
        if response not in options:
            if not pathlib.Path(response).exists() or None:
                print("Folder not found, please try again")
                continue
            else: 
                folder=pathlib.Path(response)
                break
        else: 
            folder=candidate_folders[int(response)-1]
            break
    print("You selected folder: ",folder)
    combined_file_name = folder.name
    return folder
def read_non_empty_files_from_folder(folder):
    print("Reading files in folder :",folder)
    global wdir
    wdir=os.getcwd()
    files=[f for f in pathlib.Path(folder).glob('*.csv')]
    dfs = [pd.read_csv(f,skip_blank_lines=True) for f in files if os.stat(f).st_size > 5]
    for df in dfs:
        df.columns=[c.split('.')[-1] for c in list(df.columns)]
        if v2: m = pd.to_datetime(df['Year - Quarter -  Month - Day']).dt.strftime('%Y-%m-%d')
        else: m = df['Year - Quarter - Month - Day'].astype(str)+'-'+df['Month of Year'].apply(lambda x:'{:02}'.format(x))+'-01'    
        df.insert(0, 'Month', m)
    empty_files_len=len([f for f in files if os.stat(f).st_size <= 5])
    print('{} empty files were found'.format(empty_files_len))
    print(combined_file_name)
    return dfs
def combine_dfs_from_files(dfs):
    print('combine_dfs_from_files(dfs)')
    df=pd.concat(dfs,sort=False)
    return df
def export_to_csv(df):
    print('Module export_to_csv(df)')
    rows_before=df.shape[0]
    print(df.shape[0])
    df.drop_duplicates(inplace=True)
    rows_after=df.shape[0]
    print(df.shape[0])
    print("{} duplicate rows where deteted".format(rows_before-rows_after))
    ##export to csv    
    data_folder_name = "extracted_data"
    data_folder_path = os.path.join(wdir,"extracted_data")
    os.makedirs(data_folder_path, exist_ok=True) 
    combined_file_path=os.path.join(data_folder_path,combined_file_name+'.csv')
    print('Exporting data to {}'.format(combined_file_path))
    df.to_csv(combined_file_path, index=False)

In [6]:
codes,v2=all_codes()
driver=create_driver()
init_sess(driver,v2)
# repeat(codes)

Please enter index for file in list or full path to file with code: 
Enter 1 to select all codes.txt
Enter 2 to select v3_division_monthly_F11T40.txt
Enter 3 to select v2_division_monthly_977_90.txt
Enter 4 to select .DS_Store
Enter 5 to select v3_division_monthly_total.txt
Enter 6 to select v2_division_monthly_total.txt
Enter 7 to select v2_monthly_977_90.txt
Enter 8 to select old
Enter 9 to select v2_race_ethnicity_monthly_Naxolone.txt
Enter 10 to select good
Enter 11 to select v3_division_monthly_Naxolone.txt
Enter 12 to select v3_Naxolone_Narcan.txt
Enter 13 to select v3_division_total.txt
Enter 14 to select v2_agegroup_monthly_Naxolone.txt
Enter 15 to select v2_monthly.txt
Enter 16 to select v3_region_monthly_total.txt
Enter 17 to select v2_division_monthly_Naxolone.txt
Enter 18 to select v3_race_monthly_Naxolone.txt
Enter 19 to select v3_agegroup_Naxolone.txt

 14
Parameters in code are: [['Month', '{1}-{0}-01T00:00:00']]
Please enter start month and end month a date between 2014

In [7]:
print(codes[0])

WITH SET [Medication] as FILTER
	([Medications Given].[Medication Name].allmembers,
	left([Medications Given].[Medication Name].CurrentMember.Properties("Caption"),8)="Naloxone" OR
	[Medications Given].[Medication Name].CurrentMember.Properties("Caption")="Narcan" )

Select non empty [Measures].[Count of Events] on columns,
non empty( [Medication],
[Patient Demographics].[Patient Age].[Patient Age Group],
[EMS Call Date].[Year - Quarter -  Month - Day].[Month].&[2014-01-01T00:00:00]
) ON ROWS
From [NEMSIS Enhanced]


In [ ]:
folder=pick_folder_with_files()
dfs=read_non_empty_files_from_folder(folder)
df=combine_dfs_from_files(dfs)
export_to_csv(df)

## CHALLENGES
1 - FINDING ELEMENTS

2 - SWITCHING TO IFRAMES

3 - WAITING FOR ELEMENT TO DISAPEAR

4- It looks like the clicks are remembered and are excecute at random time (found the reason for this)

5- Important to have test cases(no missing data, no duplicates , no file that reach the limit (1000 rows))


TO DO

1 - To achieve full automation, use CHromedriver option to set downloaf folder

2 - Set variables dynamically 

3- Improve user experience

4- check file, folder naming

5- Add refresh date

6- Check that  it connects to the right table (v2 or v3 by looking for table name in the page)

7- Create strategy for files with more than 1000 rows (new param such as gender just for those files?)

8- Standardize Params between two tables